In [1]:
# Import relevant libraries
import vlc
from pathlib import Path, WindowsPath
import pandas as pd
import random
from collections import deque
from tinytag import TinyTag

### Test playing a video

In [2]:
# Define parent directory for shows
TV_PATH = Path("./data/TV Shows/")

In [3]:
# Get episode 1 of show
test_vid = next(next(TV_PATH.glob("*")).glob("*"))


In [4]:
# Test playing program with VLC
media = vlc.MediaPlayer(test_vid)
media.play()

0

In [5]:
# Stop when cell is run
media.stop()

### Generate Playlist File

In [ ]:
def new_playlist_scheme(file_name : str):
    # Grab folder name for each video
    show_folders = [folder for folder in TV_PATH.iterdir()]

    # Create and populate dataframe
    show_folder_df = pd.DataFrame(columns=["show_path", "frequency"])
    show_folder_df["show_path"] = show_folders

    # Make relative to TV_PATH and remove playlist row
    show_folder_df["show_path"] = show_folder_df["show_path"].apply(lambda x: x.relative_to(TV_PATH))
    show_folder_df = show_folder_df[show_folder_df["show_path"] != WindowsPath('.playlists')]

    # Create playlist file
    show_folder_df.to_csv(TV_PATH.as_posix() + "/.playlists/" + file_name + ".csv")

In [ ]:
new_playlist_scheme("playlist")

### Test Reading Playlist File and Creating Random Subset

In [6]:
def load_playlist_scheme(file_name : str) -> pd.DataFrame:
    # Grab playlist scheme from file
    playlist = pd.read_csv(TV_PATH.as_posix() + "/.playlists/" + file_name + ".csv", index_col=0)
    return playlist

In [245]:
playlist_scheme = load_playlist_scheme("playlist")
playlist_scheme

,show_path,frequency
1,4543,10
2,Adventure Time (2010) Season 1-10 S01-S10 + Ex...,5
3,Hercules - The Legendary Journeys (1994)(tvdb-72),1
4,Star Wards - The Clone Wars (2008),20


In [184]:
def get_current_episode(show_path: Path) -> Path:
    # TODO: If .eps file does not exist, populate with first episode of show
    eps_file_path = show_path.joinpath(".eps.txt")
    if not eps_file_path.exists():
        pass
    
    # Get current episode from .eps file
    current_episode: Path = show_path.joinpath(show_path.joinpath(".eps.txt").read_text())

    return current_episode

In [93]:
def find_next_path(search_item: Path, search_path: Path) -> Path:
    '''
    Searches for the next episode or season. If the final episode or season, returns the first one
    '''
    # Search for next episode of show in folder
    sorted_paths : list[Path] = [path for path in sorted(search_path.glob("*[!.txt]"))]
    try:
        path_number : int = sorted_paths.index(search_item)
    except ValueError:
        return None
    
    # Grab the next episode if it is not the last episode
    try:
        next_path: Path = sorted_paths[path_number + 1]

    except IndexError:
        next_path : Path = sorted_paths[0]
    
    return next_path

In [739]:
def update_next_episode(show_path: Path) -> Path:
    '''
    Returns the current episode for the show. Additionally, updates the .eps file marker for the next episode. 
    If it has reached the final episode, resets to previous episode
    '''
    import os

    current_episode : Path = get_current_episode(show_path)
    next_episode : Path
    next_season : Path
    
    # If last episode and folder above is the TV_PATH, return to start of series
    if current_episode.parent.parent == TV_PATH:
        next_episode = find_next_path(current_episode, show_path)
        
        # If no next episode was found, return the current episode
        if not next_episode:
            return current_episode
        
        write_string = next_episode.as_posix().split('/')[-2:]
        write_string = write_string[0] + '/' + write_string[1]
    
    # If folder above directory is not TV_PATH, we assume this directory structure is organized into series
    # Repeat search in next season
    else:
        next_season = find_next_path(current_episode.parent, show_path)

        # If no next episode was found, return the current episode
        if not next_season:
            return current_episode
        
        # If we cannot find the next epsiode in the subsequent season folder, do not change .eps
        if os.listdir(next_season):
            return current_episode
        
        next_episode = sorted(next_season.iterdir())[0]
        write_string = next_episode.parts[-2] + '/' + next_episode.parts[-1] + next_episode.suffix
    
    # Overwrite .eps file with next episode
    with open(show_path.joinpath(".eps.txt"), "w") as file:
        file.write(next_episode.stem + next_episode.suffix)

    return next_episode

In [740]:
# Set constraints
MAX_LENGTH_MINS: int = 200

def generate_playlist(max_length: int) -> deque:
    # Define variables
    queue_length_mins: int = 0
    video_queue: deque = deque()
    selected_show: WindowsPath
    next_episode: WindowsPath

    # Generate random list of videos
    while queue_length_mins < max_length:
        # Select show from list of shows and user generated frequencies
        selected_show = random.choices(playlist_scheme["show_path"].to_list(), weights=playlist_scheme["frequency"].to_list())

        # Use .eps text file to determine next episode to play
        show_path = TV_PATH.joinpath(selected_show[0])
        next_episode = update_next_episode(show_path)
        
        # Get duration of video and append to total duration
        try:
            tag = TinyTag.get(next_episode.as_posix())
            queue_length_mins += tag.duration
        except FileNotFoundError:
            continue
        
        # Add path to video queue
        video_queue.append(next_episode)

    return video_queue


In [744]:
generate_playlist(MAX_LENGTH_MINS)

deque([WindowsPath('data/TV Shows/Star Wards - The Clone Wars (2008)/Star wars 10.mp4'),
       WindowsPath('data/TV Shows/Adventure Time (2010) Season 1-10 S01-S10 + Extras/Season 1/Adventure Time Episode 1.mp4'),
       WindowsPath('data/TV Shows/4543/file_example_MP4_480_1_5MG.mp4'),
       WindowsPath('data/TV Shows/Star Wards - The Clone Wars (2008)/Star wars 2.mp4'),
       WindowsPath('data/TV Shows/Star Wards - The Clone Wars (2008)/Star wars 1.mp4'),
       WindowsPath('data/TV Shows/Star Wards - The Clone Wars (2008)/Star wars 10.mp4'),
       WindowsPath('data/TV Shows/Star Wards - The Clone Wars (2008)/Star wars 2.mp4')])